# ACS data for Continuua of Care

In [1]:
import glob
import os
import re

import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

Sources:

- [ACS 5-Year table DP05](https://data.census.gov/cedsci/table?q=DP05%3A%20ACS%20DEMOGRAPHIC%20AND%20HOUSING%20ESTIMATES&g=0500000US06001,06037,06067,06073,41051,53033_1600000US0644000,0653000,0664000,0666000,4159000,5363000&tid=ACSDP5Y2019.DP05)
- [Los Angeles census tracts](https://data.census.gov/cedsci/table?q=DP05&text=DP05&g=0500000US06037%241400000&tid=ACSDP5Y2019.DP05) for cross-reference with Continuum of Care

In [2]:
county_acs_df = pd.read_csv(
    '../04_outputs/c01_USCB-ACS5Y-DP05-County.csv', dtype={'ACS Year': str, 'county_fips': str}
)

In [3]:
county_acs_df

,ACS Year,ACS Demographic,ACS Estimate,variable,ACS Total County/CoC Population,ACS Percent,geography,state,county_fips
0,2019,American Indian and Alaska Native,67610,one race,10098810,0.006695,Los Angeles Continuum of Care,California,06037
1,2019,Asian,1355587,one race,10098810,0.134232,Los Angeles Continuum of Care,California,06037
2,2019,Black or African American,745050,one race,10098810,0.073776,Los Angeles Continuum of Care,California,06037
3,2019,Hispanic or Latino (of any race),4603241,ethnicity,10098810,0.455820,Los Angeles Continuum of Care,California,06037
4,2019,Native Hawaiian and Other Pacific Islander,23460,one race,10098810,0.002323,Los Angeles Continuum of Care,California,06037
5,2019,Non-Hispanic White,2335886,ethnicity,10098810,0.231303,Los Angeles Continuum of Care,California,06037
6,2019,Not Hispanic or Latino,4667500,ethnicity,10098810,0.462183,Los Angeles Continuum of Care,California,06037
7,2019,White,4707974,one race,10098810,0.466191,Los Angeles Continuum of Care,California,06037
8,2019,Hispanic or Latino (of any race),371019,ethnicity,1656754,0.224000,Alameda County,California,06001
9,2019,Not Hispanic or Latino,1285735,ethnicity,1656754,0.776000,Alameda County,California,06001


In [4]:
county_acs_df.groupby('ACS Demographic')['ACS Year'].nunique()

ACS Demographic
American Indian and Alaska Native             1
Asian                                         1
Black or African American                     1
Hispanic or Latino (of any race)              1
Native Hawaiian and Other Pacific Islander    1
Non-Hispanic White                            1
Not Hispanic or Latino                        1
White                                         1
Name: ACS Year, dtype: int64

## CoC-Census geography crosswalk

Source: https://github.com/tomhbyrne/HUD-CoC-Geography-Crosswalk

In [5]:
crosswalk_df = pd.read_csv(
    '../01_inputs/USCB/county_coc_match.csv',
    encoding='ISO-8859-1',
    dtype={'county_fips': str},
)

In [6]:
crosswalk_df.rename(columns={'coc_number': 'CoC Number', 'coc_name': 'CoC Name'},inplace=True)

In [7]:
crosswalk_df['pct_cnty_pop_coc'] = crosswalk_df['pct_cnty_pop_coc']/100

Exclude counties that comprise greater than one Continuum of Care

rel_type: The relationship type of the county-CoC combination represented by each row. The relationship types are as follows:
- 1: The boundaries of a single CoC and a single county are coterminous.
- 2: A CoC's boundary fully encompass multiple counties.
- 3: A single county fully contains multiple CoCs.
- 4: A CoC is partially located in multiple counties and the counties in which it is located also contain parts of multiple CoCs.



`pct_cnty_pop_coc` : the % of the county's population that is located in the CoC

`pct_cnty_pov_coc`: the % of the county's population in poverty that is located in the CoC.

In [8]:
crosswalk_df[crosswalk_df['county_fips'] == '06037']

,county_fips,CoC Number,CoC Name,coc_name_PIT,in_2017_shapefile,in_2017_PIT,n_cocs_in_county,n_counties_in_coc,mult_county_mult_coc,rel_type,pct_cnty_pop_coc,pct_cnty_pov_coc
202,06037,CA-600,Los Angeles City & County CoC,Los Angeles City & County CoC,Yes,Yes,4.0,1.0,0.0,3.0,0.919401,91.790796
203,06037,CA-612,Glendale CoC,Glendale CoC,Yes,Yes,4.0,1.0,0.0,3.0,0.019677,1.638474
204,06037,CA-607,Pasadena CoC,Pasadena CoC,Yes,Yes,4.0,1.0,0.0,3.0,0.013660,1.228969
205,06037,CA-606,Long Beach CoC,Long Beach CoC,Yes,Yes,4.0,1.0,0.0,3.0,0.047262,5.341762


In [9]:
crosswalk_df = crosswalk_df[(crosswalk_df['n_cocs_in_county'] == 1) | (
    crosswalk_df['CoC Number'] == 'CA-600')].copy()

In [10]:
crosswalk_df['CoC Name'].fillna('', inplace=True)

### CoC-County relationships

In [11]:
crosswalk_df[
    crosswalk_df['CoC Number'].isin(
        ['CA-502', 'CA-503', 'CA-600', 'CA-601', 'OR-501', 'WA-500']
    )
][
    [
        'CoC Number',
        'CoC Name',
        'coc_name_PIT',
        'n_cocs_in_county',
        'n_counties_in_coc',
        'pct_cnty_pop_coc',
    ]
]

,CoC Number,CoC Name,coc_name_PIT,n_cocs_in_county,n_counties_in_coc,pct_cnty_pop_coc
184,CA-502,"Oakland, Berkeley/Alameda County CoC",Oakland/Alameda County CoC,1.0,1.0,1.000000
202,CA-600,Los Angeles City & County CoC,Los Angeles City & County CoC,4.0,1.0,0.919401
220,CA-503,Sacramento City & County CoC,Sacramento City & County CoC,1.0,1.0,1.000000
223,CA-601,San Diego City and County CoC,San Diego City and County CoC,1.0,1.0,1.000000
2275,OR-501,"Portland, Gresham/Multnomah County CoC",Portland-Gresham-Multnomah County CoC,1.0,1.0,1.000000
3024,WA-500,Seattle/King County CoC,Seattle/King County CoC,1.0,2.0,1.000000


In [12]:
crosswalk_df.columns

Index(['county_fips', 'CoC Number', 'CoC Name', 'coc_name_PIT',
       'in_2017_shapefile', 'in_2017_PIT', 'n_cocs_in_county',
       'n_counties_in_coc', 'mult_county_mult_coc', 'rel_type',
       'pct_cnty_pop_coc', 'pct_cnty_pov_coc'],
      dtype='object')

In [13]:
coc_acs = pd.merge(county_acs_df,
                      crosswalk_df[['county_fips', 'CoC Number']],
                      how='left',
                      on='county_fips')

In [14]:
coc_acs['ACS Demographic'].unique()

array(['American Indian and Alaska Native', 'Asian',
       'Black or African American', 'Hispanic or Latino (of any race)',
       'Native Hawaiian and Other Pacific Islander', 'Non-Hispanic White',
       'Not Hispanic or Latino', 'White'], dtype=object)

## Point-in-Time Counts

In [15]:
pitc = pd.read_csv('../04_outputs/c03_pitc.csv')

In [16]:
pitc.columns

Index(['PITC Year', 'ACS Year', 'CoC Number', 'CoC Name', 'Type of Count',
       'Total CoC Homeless', 'Total CoC Unsheltered Homeless',
       'PITC Demographic', 'demographic homeless', 'demographic unsheltered'],
      dtype='object')

In [17]:
pitc = pd.read_csv('../04_outputs/c03_pitc.csv',
                      dtype={'ACS Year': str, 'PITC Year': str})

In [18]:
pitc

,PITC Year,ACS Year,CoC Number,CoC Name,Type of Count,Total CoC Homeless,Total CoC Unsheltered Homeless,PITC Demographic,demographic homeless,demographic unsheltered
0,2020,2020,CA-502,"Oakland, Berkeley/Alameda County CoC",Sheltered-Only Count,8137.0,6312.0,Non-Hispanic/Non-Latino,6676.0,5227.0
1,2020,2020,CA-503,Sacramento City & County CoC,Sheltered-Only Count,5511.0,3900.0,Non-Hispanic/Non-Latino,4534.0,3237.0
2,2020,2020,CA-600,Los Angeles City & County CoC,Sheltered and unsheltered,63706.0,46090.0,Non-Hispanic/Non-Latino,40701.0,29364.0
3,2020,2020,CA-601,San Diego City and County CoC,Sheltered and unsheltered,7638.0,3971.0,Non-Hispanic/Non-Latino,5443.0,2872.0
4,2020,2020,OR-501,"Portland, Gresham/Multnomah County CoC",Sheltered-Only Count,4173.0,2037.0,Non-Hispanic/Non-Latino,3764.0,1861.0
...,...,...,...,...,...,...,...,...,...,...
283,2015,2019,CA-503,Sacramento City & County CoC,Sheltered and unsheltered,2659.0,948.0,Multiple Races,141.0,23.0
284,2015,2019,CA-600,Los Angeles City & County CoC,Sheltered and unsheltered,41174.0,28948.0,Multiple Races,2453.0,2099.0
285,2015,2019,CA-601,San Diego City and County CoC,Sheltered and unsheltered,8742.0,4156.0,Multiple Races,465.0,289.0
286,2015,2019,OR-501,"Portland, Gresham/Multnomah County CoC",Sheltered and unsheltered,3801.0,1887.0,Multiple Races,256.0,115.0


In [19]:
pitc['ACS Demographic'] = pitc['PITC Demographic'].replace(
    {
        'Non-Hispanic/Non-Latino': 'Not Hispanic or Latino',
        'Hispanic/Latino': 'Hispanic or Latino (of any race)',
        'American Indian or Alaska Native': 'American Indian and Alaska Native',
        'Native Hawaiian or Other Pacific Islander': 'Native Hawaiian and Other Pacific Islander'
    }
)

In [20]:
white_subset = pitc[pitc['PITC Demographic']=='White'].copy()

In [21]:
white_subset['ACS Demographic'] = 'Non-Hispanic White'

In [22]:
pitc_df = pd.concat([pitc, white_subset],ignore_index=True)

In [23]:
pitc_df.columns

Index(['PITC Year', 'ACS Year', 'CoC Number', 'CoC Name', 'Type of Count',
       'Total CoC Homeless', 'Total CoC Unsheltered Homeless',
       'PITC Demographic', 'demographic homeless', 'demographic unsheltered',
       'ACS Demographic'],
      dtype='object')

In [24]:
coc_acs.columns

Index(['ACS Year', 'ACS Demographic', 'ACS Estimate', 'variable',
       'ACS Total County/CoC Population', 'ACS Percent', 'geography', 'state',
       'county_fips', 'CoC Number'],
      dtype='object')

In [25]:
coc_acs_pitc = pd.merge(
    pitc_df, coc_acs, on=['CoC Number', 'ACS Demographic', 'ACS Year'], how='inner'
)

### Percent calculations

In [26]:
coc_acs_pitc['overall unsheltered rate'] = (
    coc_acs_pitc['Total CoC Unsheltered Homeless'] / coc_acs_pitc['ACS Total County/CoC Population']
) * 100000

In [27]:
coc_acs_pitc['overall homeless rate'] = (
    (coc_acs_pitc['Total CoC Homeless'])
    / coc_acs_pitc['ACS Total County/CoC Population']
) * 100000

In [28]:
coc_acs_pitc.head(1)

,PITC Year,ACS Year,CoC Number,CoC Name,Type of Count,Total CoC Homeless,Total CoC Unsheltered Homeless,PITC Demographic,demographic homeless,demographic unsheltered,ACS Demographic,ACS Estimate,variable,ACS Total County/CoC Population,ACS Percent,geography,state,county_fips,overall unsheltered rate,overall homeless rate
0,2019,2019,CA-502,"Oakland, Berkeley/Alameda County CoC",Sheltered and unsheltered,8022.0,6312.0,Non-Hispanic/Non-Latino,6635.0,5227.0,Not Hispanic or Latino,1285735,ethnicity,1656754,0.776,Alameda County,California,06001,380.985952,484.199827


In [29]:
coc_acs_pitc['demographic homeless rate'] = (
    coc_acs_pitc['demographic homeless']
    / coc_acs_pitc['ACS Estimate']
) * 100000

In [30]:
coc_acs_pitc['demographic unsheltered rate'] = (
    coc_acs_pitc['demographic unsheltered']
    / coc_acs_pitc['ACS Estimate']
) * 100000

In [31]:
coc_acs_pitc['demographic vs overall homeless rate'] = (
    coc_acs_pitc['demographic homeless rate']
    / coc_acs_pitc['overall homeless rate']
)

In [32]:
coc_acs_pitc['demographic vs overall unsheltered rate'] = (
    coc_acs_pitc['demographic unsheltered rate']
    / coc_acs_pitc['overall unsheltered rate']
)

In [33]:
coc_acs_pitc = coc_acs_pitc.applymap(
    lambda x: re.sub('\s{2,}', ' ', x) if type(x) is str else x
)

In [34]:
coc_acs_pitc = coc_acs_pitc.applymap(
    lambda x: x.strip() if type(x) is str else x
)

In [35]:
coc_acs_pitc['demographic pct of unsheltered'] = coc_acs_pitc['demographic unsheltered'] / \
    coc_acs_pitc['Total CoC Unsheltered Homeless']

In [36]:
coc_acs_pitc['demographic pct of all homeless'] = coc_acs_pitc['demographic homeless'] / \
    coc_acs_pitc['Total CoC Homeless']

In [37]:
coc_acs_pitc.columns

Index(['PITC Year', 'ACS Year', 'CoC Number', 'CoC Name', 'Type of Count',
       'Total CoC Homeless', 'Total CoC Unsheltered Homeless',
       'PITC Demographic', 'demographic homeless', 'demographic unsheltered',
       'ACS Demographic', 'ACS Estimate', 'variable',
       'ACS Total County/CoC Population', 'ACS Percent', 'geography', 'state',
       'county_fips', 'overall unsheltered rate', 'overall homeless rate',
       'demographic homeless rate', 'demographic unsheltered rate',
       'demographic vs overall homeless rate',
       'demographic vs overall unsheltered rate',
       'demographic pct of unsheltered', 'demographic pct of all homeless'],
      dtype='object')

In [38]:
coc_acs_pitc.columns

Index(['PITC Year', 'ACS Year', 'CoC Number', 'CoC Name', 'Type of Count',
       'Total CoC Homeless', 'Total CoC Unsheltered Homeless',
       'PITC Demographic', 'demographic homeless', 'demographic unsheltered',
       'ACS Demographic', 'ACS Estimate', 'variable',
       'ACS Total County/CoC Population', 'ACS Percent', 'geography', 'state',
       'county_fips', 'overall unsheltered rate', 'overall homeless rate',
       'demographic homeless rate', 'demographic unsheltered rate',
       'demographic vs overall homeless rate',
       'demographic vs overall unsheltered rate',
       'demographic pct of unsheltered', 'demographic pct of all homeless'],
      dtype='object')

In [39]:
coc_acs_pitc[coc_acs_pitc['variable'].isnull()]

,PITC Year,ACS Year,CoC Number,CoC Name,Type of Count,Total CoC Homeless,Total CoC Unsheltered Homeless,PITC Demographic,demographic homeless,demographic unsheltered,...,state,county_fips,overall unsheltered rate,overall homeless rate,demographic homeless rate,demographic unsheltered rate,demographic vs overall homeless rate,demographic vs overall unsheltered rate,demographic pct of unsheltered,demographic pct of all homeless


In [40]:
coc_acs_pitc['PITC Demographic'].unique()

array(['Non-Hispanic/Non-Latino', 'Hispanic/Latino', 'White',
       'Black or African American', 'Asian',
       'American Indian or Alaska Native',
       'Native Hawaiian or Other Pacific Islander'], dtype=object)

In [41]:
coc_acs_pitc['ACS Demographic'].unique()

array(['Not Hispanic or Latino', 'Hispanic or Latino (of any race)',
       'White', 'Black or African American', 'Asian',
       'American Indian and Alaska Native',
       'Native Hawaiian and Other Pacific Islander', 'Non-Hispanic White'],
      dtype=object)

In [42]:
coc_acs_pitc

,PITC Year,ACS Year,CoC Number,CoC Name,Type of Count,Total CoC Homeless,Total CoC Unsheltered Homeless,PITC Demographic,demographic homeless,demographic unsheltered,...,state,county_fips,overall unsheltered rate,overall homeless rate,demographic homeless rate,demographic unsheltered rate,demographic vs overall homeless rate,demographic vs overall unsheltered rate,demographic pct of unsheltered,demographic pct of all homeless
0,2019,2019,CA-502,"Oakland, Berkeley/Alameda County CoC",Sheltered and unsheltered,8022.0,6312.0,Non-Hispanic/Non-Latino,6635.0,5227.0,...,California,06001,380.985952,484.199827,516.047241,406.537895,1.065773,1.067068,0.828105,0.827100
1,2018,2019,CA-502,"Oakland, Berkeley/Alameda County CoC",Sheltered-Only Count,5496.0,3863.0,Non-Hispanic/Non-Latino,4567.0,3240.0,...,California,06001,233.166783,331.733015,355.205388,251.995940,1.070757,1.080754,0.838726,0.830968
2,2017,2019,CA-502,"Oakland, Berkeley/Alameda County CoC",Sheltered and unsheltered,5629.0,3863.0,Non-Hispanic/Non-Latino,4677.0,3240.0,...,California,06001,233.166783,339.760761,363.760806,251.995940,1.070638,1.080754,0.838726,0.830876
3,2016,2019,CA-502,"Oakland, Berkeley/Alameda County CoC",Sheltered-Only Count,4145.0,2397.0,Non-Hispanic/Non-Latino,3327.0,1957.0,...,California,06001,144.680502,250.188018,258.762498,152.208659,1.034272,1.052033,0.816437,0.802654
4,2015,2019,CA-502,"Oakland, Berkeley/Alameda County CoC",Sheltered and unsheltered,4040.0,2397.0,Non-Hispanic/Non-Latino,3280.0,1957.0,...,California,06001,144.680502,243.850324,255.107001,152.208659,1.046162,1.052033,0.816437,0.811881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019,2019,WA-500,Seattle/King County CoC,Sheltered and unsheltered,11199.0,5228.0,White,4742.0,2277.0,...,Washington,53033,238.123217,510.088353,362.355386,173.994773,0.710378,0.730692,0.435539,0.423431
236,2018,2019,WA-500,Seattle/King County CoC,Sheltered and unsheltered,12112.0,6320.0,White,5789.0,3308.0,...,Washington,53033,287.861273,551.673376,442.360888,252.777650,0.801853,0.878123,0.523418,0.477956
237,2017,2019,WA-500,Seattle/King County CoC,Sheltered and unsheltered,11643.0,5485.0,White,5253.0,2668.0,...,Washington,53033,249.828969,530.311519,401.402962,203.872664,0.756919,0.816049,0.486418,0.451172
238,2016,2019,WA-500,Seattle/King County CoC,Sheltered and unsheltered,10730.0,4505.0,White,4969.0,2278.0,...,Washington,53033,205.192252,488.726496,379.701374,174.071187,0.776920,0.848332,0.505660,0.463094


In [43]:
coc_acs_pitc[coc_acs_pitc['PITC Year'].isin(['2015', '2017', '2019'])].to_csv(
    '../04_outputs/c02_acs_pitc.csv', index=False
)